In [ ]:
import cv2
import pandas as pd 
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.express as px
import deeplabcut
from  triangulation import *

from mpl_toolkits.mplot3d import Axes3D
from zedStereoUtility import *

with open('selected_indices.pkl', 'rb') as file:
    saved_indices = pkl.load(file)

cup_tb_true_pos = [(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (5, 0, 0), (6, 0, 0), (7, 0, 0), 
                   (8, 0, 0), (9, 0, 0), (9, 1, 0), (9, 2, 0), (9, 3, 0), (9, 4, 0), (9, 5, 0), (9, 6, 0), 
                   (8, 6, 0), (7, 5, 0), (6, 4, 0), (5, 3, 0), (4, 2, 0), (3, 1, 0), (2, 0, 0)]
pitcher_true_pos =  [(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (5, 0, 0), (6, 0, 0), (7, 0, 0), 
                (7, 1, 0), (7, 2, 0), (7, 3, 0), (7, 4, 0), (6, 3, 0), (5, 2, 0), (4, 1, 0), (3, 0, 0)]

In [ ]:
# test_folder = '1642520848'
# video_folder = '/home/ychen/Desktop/make_tea/camera-main/videos/{}'.format(test_folder)
# config_path3d = '/home/ychen/Desktop/make_tea/dlc3D/pitcher_3d/config.yaml'
# triangulate(config_path3d, video_folder, videotype='mp4', filterpredictions=True,
#             destfolder=video_folder, save_as_csv=True)

## Model Comparison over teabag

In [ ]:
# Getting the coordinate from the dlc folder for now
object_set = 1642519365
l_tb_img_dir = './dataset/{}/left/'.format(object_set)
r_tb_img_dir = './dataset/{}/right/'.format(object_set)
tb_disp_path = './dataset/{}/disparity/'.format(object_set)

l_tb_datafile = '../camera-main/videos/{}/{}-leftDLC_dlcrnetms5_make_tea_01052022Jan5shuffle1_40000_full.pickle'.format(object_set, object_set)
r_tb_datafile = '../camera-main/videos/{}/{}-rightDLC_dlcrnetms5_make_tea_01052022Jan5shuffle1_40000_full.pickle'.format(object_set, object_set)

with open(r_tb_datafile, 'rb') as file:
    r_tb_data = pkl.load(file)
with open(l_tb_datafile, 'rb') as file:
    l_tb_data = pkl.load(file)
tb_coordinate3D = LEAStereoCoordinate(l_tb_img_dir, tb_disp_path, l_tb_data)

cup_tb_true_vect = []
cup_tb_true_dist = []
prev = False
for pos in cup_tb_true_pos:
    if prev: cup_tb_true_vect.append([pos[i] - prev[i] for i in range(len(pos))])
    prev = pos
    
for vec in cup_tb_true_vect:
    vec = [GRID_LENGTH*i for i in vec]
    cup_tb_true_dist.append(distanceCal(*vec))
    
tb_model_pos = []        
for i in tb_coordinate3D.keys():
    holder = []
    for pos in tb_coordinate3D[i][:2]:
        holder.append([pos['X'], pos['Y'], pos['Z']])
    tb_model_pos.append(holder)

tb_model_vect, tb_model_dist = getMeasurements(tb_model_pos)
# print([tb_true_dist[i] - tb_model_dist[i] for i in range(7)])
print('TB LEAstereo MSE:', mean_squared_error(cup_tb_true_dist, tb_model_dist))

xaxis_model_loss = consistencyLoss(tb_model_vect[:9])
yaxis_model_loss = consistencyLoss(tb_model_vect[9:16])
diagonal_model_loss = consistencyLoss(tb_model_vect[16:])
print('TB Axis Consistency Loss(LEAStereo):', xaxis_model_loss, yaxis_model_loss, diagonal_model_loss)

In [ ]:
tb_dlc_pos = []
tb3d_path = '../camera-main/videos/{}/{}_make_tea_3D.csv'.format(object_set, object_set)
tb3d_csv = pd.read_csv(tb3d_path)
part1 = tb3d_csv.iloc[0].tolist()
part2 = tb3d_csv.iloc[1].tolist()
part3 = tb3d_csv.iloc[2].tolist()

new_headers = formHeaders(part1, [part2, part3])
new_tb3d_csv = tb3d_csv[3:].copy()
new_tb3d_csv.columns = new_headers

new_tb3d_csv.reset_index(inplace=True)
new_tb3d_csv = new_tb3d_csv.loc[:,['ind1-head-x', 'ind1-head-y', 'ind1-head-z',
                                    'ind1-tail-x', 'ind1-tail-y', 'ind1-tail-z']]
for i in saved_indices[object_set]:
    row = [float(i) for i in new_tb3d_csv.iloc[i].tolist()]
    tb_dlc_pos.append([row[:3], row[3:6]])
    
tb_dlc_vect, tb_dlc_dist = getMeasurements(tb_dlc_pos)
scales = np.arange(1.5, 3, 0.05)
error = findError(cup_tb_true_dist, tb_dlc_dist, scales)

plt.plot(scales, error)
print('TB Min MSE:', min(error))

xaxis_dlc_loss = consistency(tb_dlc_vect[:9])
yaxis_dlc_loss = consistency(tb_dlc_vect[9:16])
diagonal_dlc_loss = consistency(tb_dlc_vect[16:])
print('TB Axis Consistency Loss(DLC):', xaxis_dlc_loss, yaxis_dlc_loss, diagonal_dlc_loss)

### 3D Trackerlet Plot for Teabag using LEAStereo

In [ ]:
tb_model_df = pd.DataFrame(np.array(tb_model_pos).reshape(-1, 3))
tb_model_df['time'] = tb_model_df.index
fig = px.scatter_3d(tb_model_df,x=0,y=1,z=2, color='time')
fig.update_traces(marker=dict(size=3))
fig.show()

### 3D Trackerlet Plot for Teabag using DLC

In [ ]:
tb_dlc_df = pd.DataFrame(np.array(tb_dlc_pos).reshape(-1, 3))
tb_dlc_df['time'] = tb_dlc_df.index
fig = px.scatter_3d(tb_dlc_df,x=0,y=1,z=2, color='time')
fig.update_traces(marker=dict(size=3))
fig.show()

In [ ]:
# with open('../dlc/make_tea_3d-Luke_Yinghan-2022-01-13-3d/camera_matrix/stereo_params.pickle', 'rb') as file:
#     params = pkl.load(file)
# outputs_l = []
# for i in tb_dlc_pos:
# #     output = np.transpose(np.dot(params['left-right']['R'], np.transpose(i)))
# #     output = [[row[j] + params['left-right']['T'][j][0] for j in range(3)] for row in output]
# #     print(output)
#     output = np.transpose(np.dot(params['left-right']['cameraMatrix1'], np.transpose(i)))
#     outputs_l.append((output[1]/output[1,2])[:2])

# outputs_r = []
# for i in tb_dlc_pos:
#     output = np.transpose(np.dot(params['left-right']['cameraMatrix2'], np.transpose(i)))
#     outputs_r.append((output[1]/output[1,2])[:2])

In [ ]:
# count = 1
# for ind, key in enumerate(tb_coordinate3D.keys()):
#     lx, ly, rx = [], [], [] 
#     dlc_ry, dlc_rx = [], [] 
#     right_coord = r_tb_data[key]['coordinates'][0]
#     for k, _ in enumerate(right_coord):
#         confid_k = list(np.reshape(r_tb_data[key]['confidence'][k], (-1)))
#         best_index = confid_k.index(max(confid_k))
#         pix = right_coord[k][best_index]
#         if len(pix) == 0: continue
#         x_l, y_l = (int(i) for dim, i in enumerate(pix))
#         dlc_ry.append(y_l)
#         dlc_rx.append(x_l)
#     for coor in tb_coordinate3D[key][:2]:
#         lx.append(coor['left_x'])
#         ly.append(coor['left_y'])
#         rx.append(coor['right_x'])
#     img_file = '{}.jpg'.format(key)
#     img_left = imageio.imread(l_tb_img_dir + img_file)
#     img_right = imageio.imread(r_tb_img_dir + img_file)

#     disp_path = "{}_disp.npy".format(tb_disp_path + img_file.split('.')[0])
#     img_disparity = np.load(disp_path)
#     fig = plt.figure(figsize=(20, 40))

#     fig.add_subplot(15, 4, count)
#     plt.title('Left Image')
#     plt.imshow(img_left)
    
#     plt.scatter(lx, ly)
    
#     fig.add_subplot(15, 4, count+1)
#     plt.title('Right Image')
#     plt.imshow(img_right)
#     plt.scatter(rx, ly)
#     plt.scatter(dlc_rx, dlc_ry)
#     count += 2
# #     fig.add_subplot(3, 1, 3)
# #     plt.title('Disparity (Left-Camerea View)')
# #     plt.imshow(img_disparity)

## Model Comparison over Cup Object

In [ ]:
# Getting the coordinate from the dlc folder for now
cup_object_set = 1642520604
l_cup_img_dir = './dataset/{}/left/'.format(cup_object_set)
r_cup_img_dir = './dataset/{}/right/'.format(cup_object_set)
cup_disp_path = './dataset/{}/disparity/'.format(cup_object_set)

l_cup_datafile = '../camera-main/videos/{}/{}-leftDLC_dlcrnetms5_cupJun15shuffle1_140000_full.pickle'.format(cup_object_set, cup_object_set)
r_cup_datafile = '../camera-main/videos/{}/{}-rightDLC_dlcrnetms5_cupJun15shuffle1_140000_full.pickle'.format(cup_object_set, cup_object_set)


with open(r_cup_datafile, 'rb') as file:
    r_cup_data = pkl.load(file)
with open(l_cup_datafile, 'rb') as file:
    l_cup_data = pkl.load(file)

cup_coordinate3D = LEAStereoCoordinate(l_cup_img_dir, cup_disp_path, l_cup_data)
cup_model_pos = []
for i in cup_coordinate3D.keys():
    holder = []
    for pos in cup_coordinate3D[i][2:5]:
        holder.append([pos['X'], pos['Y'], pos['Z']])
    cup_model_pos.append(holder)
    
cup_model_vect, cup_model_dist = getMeasurements(cup_model_pos)
# print([cup_true_dist[i] - cup_model_dist[i] for i in range(len(cup_true_dist))])
print('Cup LEAstereo MSE:', mean_squared_error(cup_tb_true_dist, cup_model_dist))

xaxis_cup_model_loss = consistency(cup_model_vect[:9])
yaxis_cup_model_loss = consistency(cup_model_vect[9:16])
diagon_cup_model_loss = consistency(cup_model_vect[16:])
print('Cup Axis Model Loss(LEAStereo):', xaxis_cup_model_loss, yaxis_cup_model_loss, diagon_cup_model_loss)

In [ ]:
cup_dlc_pos = []
cup3d_path = '../camera-main/videos/{}/{}_make_tea_3D.csv'.format(cup_object_set,cup_object_set)
cup3d_csv = pd.read_csv(cup3d_path)
part1 = cup3d_csv.iloc[0].tolist()
part2 = cup3d_csv.iloc[1].tolist()
part3 = cup3d_csv.iloc[2].tolist()

new_headers = formHeaders(part1, [part2, part3])
new_cup3d_csv = cup3d_csv[3:].copy()
new_cup3d_csv.columns = new_headers
new_cup3d_csv.reset_index(inplace=True)
new_cup3d_csv = new_cup3d_csv.loc[:,['single-cup_h-x', 'single-cup_h-y', 'single-cup_h-z',
                                    'single-cup_m-x', 'single-cup_m-y', 'single-cup_m-z',
                                    'single-cup_t-x', 'single-cup_t-y', 'single-cup_t-z']]


# new_tap3d_csv = new_tap3d_csv.drop(new_tap3d_csv.columns[[0]], axis=1) 
for i in saved_indices[cup_object_set]:
    row = [float(i) for i in new_cup3d_csv.iloc[i].tolist()]
    cup_dlc_pos.append([row[:3], row[3:6], row[6:9]])
    
cup_dlc_vect, cup_dlc_dist = getMeasurements(cup_dlc_pos)
scales = np.arange(1.5, 3, 0.05)
loss = findError(cup_tb_true_dist, cup_dlc_dist, scales)

plt.plot(scales, error)
print('Cup Min DLC MSE:', min(error))
xaxis_cup_dlc_loss = consistency(cup_dlc_vect[:9])
yaxis_cup_dlc_loss = consistency(cup_dlc_vect[9:16])
diagon_cup_dlc_loss = consistency(cup_dlc_vect[16:])
print('Cup Axis Consistency Loss(DLC):', xaxis_cup_dlc_loss, yaxis_cup_dlc_loss, diagon_cup_dlc_loss)

### 3D Trackerlet Plot for Cup using LEAStereo

In [ ]:
cup_model_df = pd.DataFrame(np.array(cup_model_pos).reshape(-1, 3))
cup_model_df['time'] = cup_model_df.index
fig = px.scatter_3d(cup_model_df,x=0,y=1,z=2,color='time')
fig.update_traces(marker=dict(size=3))
fig.show()

### 3D Trackerlet Plot for Cup using DLC

In [ ]:
cup_dlc_df = pd.DataFrame(np.array(cup_dlc_pos).reshape(-1, 3))
cup_dlc_df['time'] = cup_dlc_df.index
fig = px.scatter_3d(cup_dlc_df, x=0, y=1, z=2, color='time')
fig.update_traces(marker=dict(size=3))
fig.show()

## Model Comparison over Pitcher

In [1]:
import cv2
import pandas as pd 
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.express as px
import deeplabcut
from  triangulation import *

from mpl_toolkits.mplot3d import Axes3D
from zedStereoUtility import *

with open('selected_indices.pkl', 'rb') as file:
    saved_indices = pkl.load(file)

cup_tb_true_pos = [(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (5, 0, 0), (6, 0, 0), (7, 0, 0), 
                   (8, 0, 0), (9, 0, 0), (9, 1, 0), (9, 2, 0), (9, 3, 0), (9, 4, 0), (9, 5, 0), (9, 6, 0), 
                   (8, 6, 0), (7, 5, 0), (6, 4, 0), (5, 3, 0), (4, 2, 0), (3, 1, 0), (2, 0, 0)]
pitcher_true_pos =  [(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (5, 0, 0), (6, 0, 0), (7, 0, 0), 
                (7, 1, 0), (7, 2, 0), (7, 3, 0), (7, 4, 0), (6, 3, 0), (5, 2, 0), (4, 1, 0), (3, 0, 0)]

Loading DLC 2.2.1.1...


/home/ychen/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/deeplabcut/__init__.py:81: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


In [27]:
## Getting the coordinate from the dlc folder for now
pitcher_object_set = 1642520848
l_pitcher_img_dir = './dataset/{}/left/'.format(pitcher_object_set)
r_pitcher_img_dir = './dataset/{}/right/'.format(pitcher_object_set)
pitcher_disp_path = './dataset/{}/disparity/'.format(pitcher_object_set)

l_pitcher_datafile = '../camera-main/videos/{}/{}-leftDLC_dlcrnetms5_make_tea_revised_pitcherJun14shuffle1_50000_full.pickle'.format(pitcher_object_set, pitcher_object_set)
r_pitcher_datafile = '../camera-main/videos/{}/{}-rightDLC_dlcrnetms5_make_tea_revised_pitcherJun14shuffle1_50000_full.pickle'.format(pitcher_object_set, pitcher_object_set)

with open(r_pitcher_datafile, 'rb') as file:
    r_pitcher_data = pkl.load(file)
with open(l_pitcher_datafile, 'rb') as file:
    l_pitcher_data = pkl.load(file)

    
pitcher_true_vect = []
pitcher_true_dist = []
prev = False
for pos in pitcher_true_pos:
    if prev: pitcher_true_vect.append([pos[i] - prev[i] for i in range(len(pos))])
    prev = pos
    
for vec in pitcher_true_vect:
    vec = [GRID_LENGTH*i for i in vec]
    pitcher_true_dist.append(distanceCal(*vec))
    
pitcher_coordinate3D = LEAStereoCoordinate(l_pitcher_img_dir, pitcher_disp_path, l_pitcher_data)

# keep only the points that are alway shown in all frames
tracked_pitcher_pts = [True for _ in range(4)]
for fid, obj_pts in pitcher_coordinate3D.items():
    for k, pt in enumerate(obj_pts):
        if pt==False:
            tracked_pitcher_pts[k] = False      

# extract the points from pitcher.
pitcher_model_pos = []
for i in pitcher_coordinate3D.keys():
    holder = []
    for p, pos in enumerate(pitcher_coordinate3D[i]):
        if p==1:
            holder.append([pos['X'], pos['Y'], pos['Z']])
    pitcher_model_pos.append(holder)
    
    
pitcher_model_vect, pitcher_model_dist = getMeasurements(pitcher_model_pos)
# print([pitcher_true_dist[i] - pitcher_model_dist[i] for i in range(len(pitcher_true_dist))])
print('Pitcher LEAstereo MSE:', mean_squared_error(pitcher_true_dist, pitcher_model_dist))

xaxis_pitcher_model_loss = consistency(pitcher_model_vect[:7])
yaxis_pitcher_model_loss = consistency(pitcher_model_vect[7:11])
diagon_pitcher_model_loss = consistency(pitcher_model_vect[11:])
print('Pitcher Axis Consistency Loss(LEAStereo):', xaxis_pitcher_model_loss, yaxis_pitcher_model_loss, diagon_pitcher_model_loss)

/home/ychen/Desktop/make_tea/LEAstereo/zedStereoUtility.py:88: RuntimeWarning:

divide by zero encountered in double_scalars



Pitcher LEAstereo MSE: 0.21395992049706417
Pitcher Axis Consistency Loss(LEAStereo): 0.04478024602485481 0.10589606788376879 0.044393701524836106


In [36]:
pitcher_model_vect

[[[4.632152622068839, 0.33133962381282345, -0.4004264919114888]],
 [[4.7918746511161565, 0.05275892244997138, -0.8887387852759119]],
 [[5.059386377430593, -0.25349501517031925, 0.0014994189925445767]],
 [[4.739095596863069, 0.04032214671923029, -0.6480339710983358]],
 [[4.637760312459453, 0.17509402841233523, -0.3177931018169673]],
 [[5.2704988230906, -0.0934228291989605, -0.08744979026872102]],
 [[4.782787721158417, 0.014234560722414358, -0.5593444098984506]],
 [[-0.5809787376229743, 3.591274730195533, -2.0353760224198325]],
 [[-0.5269322225649091, 4.23061763283801, -2.8352957031715107]],
 [[-0.14270191995799664, 3.7832604965258003, -2.250879934580979]],
 [[0.200832505473155, 3.9818493263014014, -0.9155819377568477]],
 [[-5.184301146656248, -4.120651822974865, 1.6347658583447284]],
 [[-4.231062411605251, -3.6362129465869852, 2.1824389086087947]],
 [[-4.364306606808215, -4.218150177170086, 2.697778172844451]],
 [[-4.725033604416531, -3.8405798380793534, 3.217268780945389]]]

In [35]:
pitcher_dlc_vect

[[[8.973503353813896, -1.059597354821598, 4.538986753396188]],
 [[3.785518683979973, 0.4547618108970042, -2.9154104367597]],
 [[5.4830854648301965, 0.2045000442380669, -0.8340634383769157]],
 [[8.692670890523914, -1.026089640598478, 2.593982857900855]],
 [[7.026757724852402, -0.2573215266907205, 1.049945684695757]],
 [[5.857006313884398, 0.15174770469713295, -0.9770900489280194]],
 [[5.688542312081154, -0.02250630954778643, -0.7707463156145451]],
 [[-6.9122381713977745, 4.611366615494401, -3.919979734425908]],
 [[-11.370070107463548, 5.8872438813986, -8.139982971318204]],
 [[-5.230286253490121, 4.698318186472807, -2.7631919074695332]],
 [[-8.953940491850176, 5.212113034619855, -6.052914185130206]],
 [[3.255910055023321, -5.38696361399409, 6.821620294448152]],
 [[-0.7745596875564758, -4.4164169843494445, 2.418948870270654]],
 [[0.8326522177308391, -4.779829241324098, 3.7546152494264646]],
 [[6.008755873366752, -6.025805522474126, 10.634240909441559]]]

In [37]:
pitcher_dlc_dist

[10.111820298183718,
 4.799643531157202,
 5.549928675394186,
 9.129300861982195,
 7.109425042342214,
 5.939886807884196,
 5.740563565830489,
 9.187490392862163,
 15.172259473824317,
 7.554158959956692,
 11.999039242781016,
 9.28196265379931,
 5.094702672994911,
 6.134917518853431,
 13.619932417616452]

In [38]:
pitcher_dlc_pos = []
pitcher3d_path = '../camera-main/videos/{}/{}_pitcher3d.csv'.format(pitcher_object_set, pitcher_object_set)
pitcher3d_csv = pd.read_csv(pitcher3d_path)
part1 = pitcher3d_csv.iloc[0].tolist()
part2 = pitcher3d_csv.iloc[1].tolist()
part3 = pitcher3d_csv.iloc[2].tolist()

new_headers = formHeaders(part1, [part2, part3])
new_pitcher3d_csv = pitcher3d_csv[3:].copy()
new_pitcher3d_csv.columns = new_headers

new_pitcher3d_csv.reset_index(inplace=True)
new_pitcher3d_csv = new_pitcher3d_csv.loc[:,['pitcher-tip-x', 'pitcher-tip-y', 'pitcher-tip-z',
                                             'pitcher-top_handle_inner-x', 'pitcher-top_handle_inner-y', 'pitcher-top_handle_inner-z',
                                             'pitcher-top_handle_outer-x', 'pitcher-top_handle_outer-y', 'pitcher-top_handle_outer-z',
                                             'pitcher-bottom_handle_outer-x', 'pitcher-bottom_handle_outer-y', 'pitcher-bottom_handle_outer-z']]
for i in saved_indices[pitcher_object_set]:
    row = [float(i) for i in new_pitcher3d_csv.iloc[i].tolist()]
    pitcher_dlc_pos.append([[j/10 for j in row[3:6]]])
    
pitcher_dlc_vect, pitcher_dlc_dist = getMeasurements(pitcher_dlc_pos)

print('Pitcher LEAstereo MSE:', mean_squared_error(pitcher_true_dist, pitcher_dlc_dist))
print('Pitcher DLC3D MSE:', min(error))

xaxis_dlc_loss = consistency(pitcher_dlc_vect[:7])
yaxis_dlc_loss = consistency(pitcher_dlc_vect[7:11])
diagonal_dlc_loss = consistency(pitcher_dlc_vect[11:])
print('Pitcher Axis Consistency Loss(DLC):', xaxis_dlc_loss, yaxis_dlc_loss, diagonal_dlc_loss)

Pitcher LEAstereo MSE: 18.872531078671194
Pitcher DLC3D MSE: 18.87253107867119
Pitcher Axis Consistency Loss(DLC): 1.324675103154159 0.17439497472552248 0.6833519947626949


### 3D Trackerlet Plot for Pitcher using LEAStereo

In [39]:
import plotly.express as px
pitcher_model_df = pd.DataFrame(np.array(pitcher_model_pos).reshape(-1, 3))
pitcher_model_df['time'] = pitcher_model_df.index
fig = px.scatter_3d(pitcher_model_df,x=0,y=1,z=2,color='time')
fig.update_traces(marker=dict(size=3))
fig.show()

### 3D Trackerlet Plot for Pitcher using DLC

In [40]:
pitcher_dlc_df = pd.DataFrame(np.array(pitcher_dlc_pos).reshape(-1, 3))
pitcher_dlc_df['time'] = pitcher_dlc_df.index
fig = px.scatter_3d(pitcher_dlc_df, x=0, y=1, z=2, color='time')
fig.update_traces(marker=dict(size=3))
fig.show()

In [ ]:
# count = 1

# for ind, key in enumerate(cup_coordinate3D.keys()):
#     lx, ly, rx = [], [], [] 
#     dlc_ry, dlc_rx = [], [] 
#     right_coord = r_cup_data[key]['coordinates'][0]
# #     for pix in right_coord:
# #         if len(pix) == 0: continue
# #         x_l, y_l = (int(i) for dim, i in enumerate(pix[0]))
# #         dlc_ry.append(y_l)
# #         dlc_rx.append(x_l)
#     for coor in cup_coordinate3D[key][2:5]:
#         lx.append(coor['left_x'])
#         ly.append(coor['left_y'])
#         rx.append(coor['right_x'])
#     img_file = next(filename for filename in os.listdir(l_cup_img_dir) 
#                     if int(filename.split('.')[0][5:])==int(key.split('.')[0][5:]))
#     img_left = imageio.imread(l_cup_img_dir + img_file)
#     img_right = imageio.imread(r_cup_img_dir + img_file)

#     disp_path = "{}_disp.npy".format(cup_disp_path + img_file.split('.')[0])
#     img_disparity = np.load(disp_path)
#     fig = plt.figure(figsize=(20, 40))

#     fig.add_subplot(5, 4, count)
#     plt.title('Left Image')
#     plt.imshow(img_left)
#     plt.scatter(lx, ly)
#     fig.add_subplot(5, 4, count+1)
#     plt.title('Right Image')
#     plt.imshow(img_right)
#     plt.scatter(rx, ly)
# #     plt.scatter(dlc_rx, dlc_ry)
#     count += 2
# #     fig.add_subplot(3, 1, 3)
# #     plt.title('Disparity (Left-Camerea View)')
# #     plt.imshow(img_disparity)

In [ ]:
# # combined leastereo distances
# combine_true_dist = tb_true_dist + tap_true_dist + cup_true_dist
# combine_model_dist = tb_model_dist + tap_model_dist + cup_model_dist
# combine_dlc_dist = tb_dlc_dist + tap_dlc_dist + cup_dlc_dist

# scales = np.arange(2, 3, 0.05)
# combine_model_mse = mean_squared_error(combine_true_dist, combine_model_dist)
# combine_dlc_mse = findLosses(combine_true_dist, combine_dlc_dist, scales)

# plt.plot(scales, combine_dlc_mse)
# print('Combine Min LEAstereo MSE:', combine_model_mse)
# print('Combine Min DLC MSE:', min(combine_dlc_mse))

In [ ]:

# count=0
# extra_path = '../camera-main/videos/1642519365/1642519365-left.mp4'
# vidcap = cv2.VideoCapture(extra_path)
# success, first_frame = vidcap.read()
# while success:
# #     cv2.imwrite(os.path.join(l_img_dir, "frame{}.jpg".format(str(count).zfill(2))), image)     # save frame as JPEG file      
#     success, image = vidcap.read()
#     print('Read a new frame: ', success, count)
#     cv2.imshow("Input", image)
#     cv2.waitKey(0)
#     count += 1